In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.environ.get("OPEN_METEO_API")

In [13]:
from typing import Optional, Annotated, Union, Tuple
from datetime import datetime
import requests


def get_coordinates(city: str, country: Optional[str] = None) -> Union[Tuple[float, float], str]:

    url = "http://api.openweathermap.org/geo/1.0/direct"
    query = f"{city},{country}" if country else city
    params = {
        "q": query,
        "limit": 1,
        "appid": API_KEY
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code != 200:
            return f"Response: {response.text}"

        data = response.json()
        if not data:
            return f"Error: City '{query}' not found."
        return data[0]['lat'], data[0]['lon']

    except Exception as e:
        return f"Geocoding error: {e}"


def get_monthly_average(
    city: Annotated[str, "The city name."],
    month: Annotated[int, "The month number (1-12)."],
    country: Annotated[str, "The country (optional)."] = ""
) -> str:
    
    coords = get_coordinates(city, country)
    if not coords:
        return f"Error: Could not find coordinates for city: {city}"
    
    lat, lon = coords

    url = "https://history.openweathermap.org/data/2.5/aggregated/month"
    params = {
        "lat": lat,
        "lon": lon,
        "month": month,
        "appid": API_KEY
    }
    
    try:
        res = requests.get(url, params=params, timeout=10)

        res.raise_for_status()
        data = res.json()
        result = data.get('result', {})
        
        temp_mean = result.get('temp', {}).get('mean', 'N/A')
        precip_mean = result.get('precipitation', {}).get('mean', 'N/A')
        
        return (f"Averages for Month {month} in {city}:\n"
                f"Mean Temp: {temp_mean} K\n"
                f"Mean Precipitation: {precip_mean} mm")

    except Exception as e:
        return f"Error fetching averages: {str(e)}"

In [14]:
get_monthly_average("London", 12)

'Averages for Month 12 in London:\nMean Temp: 280.13 K\nMean Precipitation: 0.11 mm'